In [129]:
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, LSTM
import warnings
warnings.filterwarnings("ignore")

In [2]:
returns = pd.read_pickle("./Data/returns.pkl")
returns = returns.iloc[1:]

In [3]:
# dropping columns with all NaN
drop_columns = []

for col in returns.columns:
    if returns[col].isnull().all() == True:
        drop_columns.append(col)
        
returns.drop(columns=drop_columns, inplace=True)

In [4]:
def get_investable(t, n_rows):
    "Find stocks in investable universe at time t\
    (stocks in the S&P500 that have prices recorded for the last n_rows days)"
    
    df_investable = returns.copy(deep = True).sort_index(ascending = False)
    
    t_index = df_investable.index.get_loc(t)
    
    #take n_rows worth of data upto time specified
    df_investable = df_investable.iloc[t_index + 1:t_index + n_rows + 1]
    
    #find all stocks that exist in the S&P at this time period
    investable_universe = []
    for col in df_investable.columns:
        if ~df_investable[col].iloc[:n_rows].isna().any():
            investable_universe.append(col)
        
    df_investable = df_investable[investable_universe]
    
    return df_investable

In [100]:
def format_investable(df, n_in=1, n_out=1, dropnan = True):
    "Takes investable dataframe and formats it to have all stock returns at time t-1\
    (the features) and all corresponding stock returns at time t (the targets)"
    n_vars = df.shape[1]
    
    cols = []
    input_col_names = df.add_suffix(' (t-1)').columns
    forecast_col_names = df.add_suffix(' (t)').columns
    names = input_col_names.append(forecast_col_names)
    
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))

    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        
    # put it all together
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    
    # drop the first row (it contains NaNs)
    agg.dropna(inplace=True)
    
    return agg

In [126]:
def reshape_investable(df):
    "Takes formatted investable and reshapes inputs it into 3D array [samples, timesteps, features]"
    
    # select columns ending with ' (t-1)' for features
    # and columns ending with ' (t)' for targets
    X_col = [df.columns[i] for i in range(df.shape[1]) if df.columns[i].endswith(' (t-1)')]
    y_col = [df.columns[i] for i in range(df.shape[1]) if df.columns[i].endswith(' (t)')]
    
    X = df[X_col].values
    y = df[y_col].values
    
    reshaped_X = X.reshape(X.shape[0], 1, X.shape[1])
    
    return reshaped_X, y

In [71]:
t = pd.to_datetime('2020-05-22')
inv = get_investable(t, 360)

In [102]:
f_inv = format_investable(inv)

In [131]:
reshaped_X, y = reshape_investable(f_inv)

In [132]:
model = Sequential()
model.add(LSTM(50, input_shape=(reshaped_X.shape[1], reshaped_X.shape[2])))
model.add(Dense(y.shape[1]))

In [133]:
model.compile(loss = 'mae', optimizer = 'adam')

In [134]:
history = model.fit(reshaped_X, y, epochs=50, batch_size=16)

Epoch 1/50
23/23 [==============================] - 12s 2ms/step - loss: 2.0905
Epoch 2/50
23/23 [==============================] - 0s 2ms/step - loss: 1.7718
Epoch 3/50
23/23 [==============================] - 0s 2ms/step - loss: 1.8526
Epoch 4/50
23/23 [==============================] - 0s 2ms/step - loss: 1.7722
Epoch 5/50
23/23 [==============================] - 0s 2ms/step - loss: 1.7934
Epoch 6/50
23/23 [==============================] - 0s 2ms/step - loss: 1.6641
Epoch 7/50
23/23 [==============================] - 0s 2ms/step - loss: 1.6513
Epoch 8/50
23/23 [==============================] - 0s 2ms/step - loss: 1.6466
Epoch 9/50
23/23 [==============================] - 0s 2ms/step - loss: 1.7127
Epoch 10/50
23/23 [==============================] - 0s 2ms/step - loss: 1.5263
Epoch 11/50
23/23 [==============================] - 0s 2ms/step - loss: 1.4800
Epoch 12/50
23/23 [==============================] - 0s 2ms/step - loss: 1.4557
Epoch 13/50
23/23 [=============================